In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib_fontja # グラフに日本語を表示するのに必要
import nashpy as nash
from pokemon_iyasi1on1.db import get_species
from pokemon_iyasi1on1.model import NatureTarget, Poke, PokeBreeding, PokeStrategy

In [ ]:
import matplotlib
matplotlib.rc('pdf', fonttype=42) # フォント埋め込みをする

In [ ]:
ds = pickle.load(open("data/monte_carlo_win_table.pkl","rb"))

In [ ]:
n_breedings = len(ds["breedings"])

In [ ]:
# 勝率テーブル作成
winrates = np.zeros((n_breedings, n_breedings))
for match, winrate in ds["results"]:
    # 上三角だけ情報が入っている
    winrates[match] = winrate
winrates += np.tril(1.0 - winrates.T, k=-1) # 下三角を作成、k=-1: 対角成分を除去
winrates += np.eye(n_breedings) * 0.5 # 自分自身は勝率0.5

In [ ]:
# 利得行列に変換(-1~1)
payoffs = (winrates - 0.5) * 2

In [ ]:
%%time
rps = nash.Game(payoffs)
strt, _ = rps.linear_program() # 2プレイヤーの戦略が出るが対称なので1つだけ取得

In [ ]:
def breedings_str(b):
    s = f"{get_species(b.no).name.replace('のすがた','')}"
    match b.strategy:
        case PokeStrategy.VEST:
            s += ",チョッキ"
        case PokeStrategy.LEFTOVER:
            s += ",たべのこし"
    s += f",H{b.ev_hp},A{b.ev_a},B{b.ev_b},S{b.ev_s}"
    match b.nature_up:
        case NatureTarget.A:
            s += ",A↑"
        case NatureTarget.B:
            s += ",B↑"
        case NatureTarget.S:
            s += ",S↑"
    match b.nature_down:
        case NatureTarget.A:
            s += ",A↓"
        case NatureTarget.B:
            s += ",B↓"
        case NatureTarget.S:
            s += ",S↓"
    if b.ev_s == 0 and b.iv_s == 0 and b.nature_down == NatureTarget.S:
        s += ",最遅"
    return s

In [ ]:
# ナッシュ均衡で選出確率が正の値になった構築を表示
for idx in np.argsort(-strt):
    if strt[idx] <= 0.0:
        continue
    s = breedings_str(ds["breedings"][idx])
    print(f"{strt[idx]*100:.2f}%, {s}")

In [ ]:
strt_str_list = []
strt_idxs = []
for idx in np.argsort(-strt):
    if strt[idx] <= 0.0:
        continue
    strt_idxs.append(idx)
    s = breedings_str(ds["breedings"][idx])
    strt_str_list.append(s)

In [ ]:
def display_heatmap(cmap="coolwarm", savefig=None):
    
    # 戦略の数
    n = len(strt_idxs)
    
    # 空の利得行列を作成
    payoff_matrix = payoffs[strt_idxs][:, strt_idxs]
    
    # ヒートマップを描画
    fig, ax = plt.subplots()
    cax = ax.matshow(payoff_matrix, cmap=cmap)
    plt.colorbar(cax)
    
    # 軸のラベルを設定
    ax.set_xticks(range(n))
    ax.set_yticks(range(n))
    ax.set_xticklabels(strt_str_list, rotation=90, ha="left")
    ax.set_yticklabels(strt_str_list)
    
    # 数値ラベルを追加
    for i in range(n):
        for j in range(n):
            if cmap == "coolwarm":
                color = "black"
            else:
                color = 'black' if payoff_matrix[i, j] > -0.1 else 'white'
            # 見づらいので消した
            # ax.text(j, i, f"{payoff_matrix[i, j]:.3f}", ha='center', va='center', color=color)
    
    plt.title("Payoff Matrix of Strategies")
    if savefig:
        plt.savefig(savefig, bbox_inches='tight')
    plt.show()


In [ ]:
display_heatmap(savefig="payoff_nash_strategies.png")

In [ ]:
# 印刷用に追加した機能

In [ ]:
from typing import Dict, Iterable, List, Tuple
from pathlib import Path
from matplotlib import patheffects as pe


def save_figure(fig: plt.Figure, out_base: Path):
    out_base.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_base.with_suffix(".png"), dpi=200, bbox_inches="tight")
    fig.savefig(out_base.with_suffix(".pdf"), bbox_inches="tight")
    plt.close(fig)


def _lighten_cmap(base_cmap: str, amount: float = 0.35):
    """Return a lightened version of the given cmap name.

    amount in [0,1]: 0 no change, 1 becomes fully white.
    """
    cmap = plt.get_cmap(base_cmap)
    samples = cmap(np.linspace(0, 1, 256))
    rgb = samples[:, :3]
    rgb = rgb + (1.0 - rgb) * amount
    samples[:, :3] = np.clip(rgb, 0, 1)
    from matplotlib.colors import LinearSegmentedColormap

    return LinearSegmentedColormap.from_list(f"{cmap.name}_light", samples)


def plot_payoff_heatmap(
    matrix: np.ndarray, labels: List[str], title: str, color: bool, out_base: Path
):
    n = matrix.shape[0]
    # size heuristic: base 1.0 inch per cell capped
    cell = 0.45 if n <= 18 else 0.35
    w = max(6.0, min(14.0, cell * n + 3))
    h = max(5.0, min(14.0, cell * n + 3))

    fig, ax = plt.subplots(figsize=(w, h))
    vlim = max(1.0, float(np.nanmax(np.abs(matrix))))
    base = "RdBu_r" if color else "Greys"
    cmap = _lighten_cmap(base, amount=0.35)
    norm = plt.Normalize(vmin=-vlim, vmax=vlim)
    im = ax.imshow(matrix, cmap=cmap, norm=norm)

    # Annotate values if small enough
    if n <= 25:
        for i in range(n):
            for j in range(n):
                val = matrix[i, j]
                r, g, b, _ = cmap(norm(val))
                # Perceived luminance (sRGB):
                luminance = 0.2126 * r + 0.7152 * g + 0.0722 * b
                txt_color = "black" if luminance > 0.55 else "white"
                stroke = "white" if txt_color == "black" else "black"
                txt = ax.text(
                    j,
                    i,
                    f"{val:.1f}",
                    ha="center",
                    va="center",
                    fontsize=9,
                    color=txt_color,
                )
                txt.set_path_effects([pe.withStroke(linewidth=1.2, foreground=stroke)])

    ax.set_xticks(np.arange(n), labels=labels, rotation=90, ha="left")
    ax.xaxis.tick_top()
    ax.set_yticks(np.arange(n), labels=labels)
    ax.set_xlabel("Opponent party")
    ax.set_ylabel("Our party")
    # ax.set_title(title)
    cbar = fig.colorbar(im, ax=ax, shrink=0.85)
    cbar.set_label("Payoff")
    fig.tight_layout()
    save_figure(fig, out_base)

In [ ]:
matrix = payoff_matrix = payoffs[strt_idxs][:, strt_idxs]

In [ ]:
plot_payoff_heatmap(matrix, strt_str_list, "", True, Path("payoff_nash_strategies_color"))
plot_payoff_heatmap(matrix, strt_str_list, "", False, Path("payoff_nash_strategies_mono"))